In [1]:
import os, os.path
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import deque
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from utils import generation_2dbox,whole_upleft, box_cornel, feasible_location,action_options_list
config = tf.compat.v1.ConfigProto(device_count={'GPU':1})
sess = tf.compat.v1.Session(config=config) 

# 상태가 입력, 큐함수가 출력인 인공신경망 생성
class DQN(tf.keras.Model):
    def __init__(self, state_size):
        super(DQN, self).__init__()
        self.dnn1 = Dense(512, activation='relu',input_shape=state_size)#
        self.dnn2 = Dense(512, activation='relu')#
        self.dnn3 = Dense(512, activation='relu')#
        self.fc = Dense(512, activation='relu')
        self.fc_out = Dense(1)

    def call(self, x):
        x = self.dnn1(x)
        x = self.dnn2(x)
        x = self.dnn3(x)
        x = self.fc(x)
        q = self.fc_out(x)
        return q


# 브레이크아웃 예제에서의 DQN 에이전트
class DQNAgent:
    def __init__(self, state_size=(20*20+20*20,), model_path='save_model'):
        self.state_size = state_size
        self.epsilon = 0.02
        self.model = DQN(state_size)
        self.model.load_weights(model_path)
        
    def get_action(self, history,a_ops):
        if len(a_ops)==0:
            return 0,-1
        q_value_list=[]
        history = np.array([history.flatten()]*len(a_ops))
        actions = np.array([sample.flatten() for sample in a_ops])
        q_value_list=self.model(np.concatenate([history,actions],axis=1))
        
        if np.random.rand() <= self.epsilon:
            random_action_idx=random.randrange(len(a_ops))
            random_action=a_ops[random_action_idx]
            return random_action,random_action_idx
        else:
            amax_action_idx=np.argmax(q_value_list)
            amax_action=a_ops[amax_action_idx]
            return amax_action,amax_action_idx

In [ ]:
import gym
import gym_bpp
env = gym.make('bpp-v0')

model_path = 'save_model/model'
agent = DQNAgent((20*20+20*20,), model_path)
num_episode = 100

np.random.seed(seed=10011)
boxes_eps,gt_upleft_eps=generation_2dbox(N_epi=num_episode,c_l=20,c_b=20)#N,2

history_eps=[]
used_boxes_eps=[]
for e in range(num_episode):
    gt_upleft=gt_upleft_eps[e]
    boxes=boxes_eps[e]
    done = False
    step = 0
    env.reset()# env 초기화
    history = np.zeros((20,20))#container
    upleft_list=[]
    #tmp=0 
    used_boxes=[]
    
    while not done:
        step += 1
        box=boxes[step-1]
        w_upleft =whole_upleft(*box_cornel([0,0],20,20),box[0],box[1])
        for i,ul in enumerate(upleft_list):
            w_upleft += whole_upleft(*box_cornel([ul[0],ul[1]], used_boxes[i][0], used_boxes[i][1]),box[0],box[1])
        f_upleft = feasible_location(history,w_upleft,box[0],box[1])
        a_ops=action_options_list(f_upleft)
        action,action_idx = agent.get_action(history,a_ops)
        
        # 중박스 스킵 허용
        if action_idx==-1 and step != len(boxes): continue
        elif action_idx==-1 or step==len(boxes):#(action이 없거나 박스가 다 사용되거나)-----------
            print('no action or all boxes are used',np.sum(history))
            done=True
        else:
            used_boxes.append(box)
            upleft=f_upleft[action_idx]
            upleft_list.append(upleft)
            next_history = env.step(upleft,box[0],box[1])
            history = next_history
            
        #tmp+=box[0]*box[1]
        #print(np.sum(history),box[0]*box[1],box[0],box[1],tmp)
        if done:
            log = "============episode: {:5d} | ".format(e)
            log += "env.terminal_reward(): {:.3f} | ".format(env.terminal_reward())
            print(log)
            agent.avg_loss = 0
        
    history_eps.append(history)
    used_boxes_eps.append(used_boxes)

In [ ]:
pred_disp=np.zeros((20,20))
for i in range(len(upleft_list)):
    pred_disp[upleft_list[i][0]:upleft_list[i][0]+used_boxes[i][0],upleft_list[i][1]:upleft_list[i][1]+used_boxes[i][1]] += 0.5
# plt.imshow(pred_disp,cmap='tab20')
plt.imshow(pred_disp)

In [ ]:
#for i in history_list:
#    print(np.mean(i))
fracs=[np.mean(i) for i in history_eps]
plt.plot(fracs,'.-')
plt.grid()

In [ ]:
fig, axs = plt.subplots(2,6, figsize=(15, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()

i=0
for tmp in history_eps[:12]:
    axs[i].imshow(tmp);i+=1
    #fig.add_subplot(()).imshow(tmp)
    #fig.grid()